In [2]:
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import tweepy
import json
import pandas as pd
import csv
import re
from textblob import TextBlob
import string
import preprocessor as p
import os
import time

#### Scraping data using tweepy+Twitter API:

In [3]:
# Twitter credentials
#Authentication
consumer_key = ''
consumer_secret = ''
access_key = ''
access_secret = ''

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)

#### Searching for tweets via twitter query:

In [4]:
text_query = '$TSLA min_replies:0 min_faves:2 -filter:retweets'
max_tweets = 3200
 
# query for Twitter API
tweets = tweepy.Cursor(api.search,q=text_query).items(max_tweets)
 
# Pulling information from tweets iterable object
#Specifying columns:
tweets_list = [[tweet.text, tweet.created_at, tweet.id_str, tweet.user.name, tweet.user.screen_name, tweet.user.id_str, tweet.user.location, tweet.user.url, tweet.user.description] for tweet in tweets]
 
# Creation of dataframe from tweets_list
# Did not include column names to simplify code 
tweets_df = pd.DataFrame(tweets_list)

In [5]:
tweets_df

,0,1,2,3,4,5,6,7,8
0,保有状況11/14\n$PLUG 60株(+50)\n$V 58株\n$MA 38株\n$T...,2020-11-14 10:46:37,1327563581529882626,da_cider,da_cider_gbf,809626003467177985,日本 沖縄,None,公務員(30代後半)、 $V $MA $FB $TSLA $PLUG $ZM 米国株投資中。
1,@1stPrincipleInv @elonmusk if you’re not gobbl...,2020-11-14 10:24:32,1327558025813569536,stekkerauto 🚗⚡️,stekkerauto,897596339227373569,"Berlin, Germany",None,Emobility & renewable energy ⚡️ „if you dislik...
2,Valores para el Repaso Semanal Orden de Aparic...,2020-11-14 09:53:59,1327550337323065346,Enric Buqué,Buguisim,2994946468,,https://t.co/scyLaluYxe,Fundador del Canal de YouTube Bolsa Free!!! Bu...
3,Haha WOW #FSDBeta just drove me from the front...,2020-11-14 09:52:36,1327549989866729472,Eli,EliBurton_,1060049746713182208,"California, USA",https://t.co/uqoEELz4Q1,#GirlDad Dedicated to inspiring future martian...
4,現在の保有銘柄【米国株】\n$BNTX $CRWD $SHOP $TSLA $ZM $SQ ...,2020-11-14 09:48:30,1327548958160285699,ハギ🐠兼業投資家,hagihagi09,1241750341097381888,日本 東京,https://t.co/lmls6Ng6pF,投資歴8年程の兼業投資家。日本株、米国株などをポチポチやりながら、勝てる投資法を探り中。
...,...,...,...,...,...,...,...,...,...
3195,$GWSO : \n\n#ElectricVehicles #Globalwarming #...,2020-11-10 15:56:51,1326192102321999873,☀️SUNNYLAND☀️,SUNNYLAND24,790440452310704128,"Washington, USA",https://t.co/fV6RIizLA5,Army Vet. Family Man. 13 Year OTC Stock Invest...
3196,A lot of margin calls today at Robinhood: $tsl...,2020-11-10 15:56:22,1326191979659751425,Dave Kranzler,InvResDynamics,66649220,Denver,http://t.co/ksEUfdoGRX,"""In times of universal deceit, telling the tru..."
3197,Technically as long as $PLTR doesnt drop below...,2020-11-10 15:56:04,1326191906397835264,H.Ankhs,MemphistoPhele1,2518258765,🚪,None,Explicitly attempted world domination. Got boo...
3198,Even if you have less than a year experience o...,2020-11-10 15:55:50,1326191847497113600,INVESTOR K,radioactinium,110591829,,None,1. Cryptocurrency 2. Stock 3. Gold


#### Retrieving tweets with proper place format:

In [13]:

tweets_df.loc[tweets_df[6] != ''][6]

0                          日本 沖縄
1                Berlin, Germany
3                California, USA
4                          日本 東京
5       Amsterdam, Istanbul, NYC
                  ...           
3193                    New York
3194            Brossard, Québec
3195             Washington, USA
3196                      Denver
3197                           🚪
Name: 6, Length: 2040, dtype: object

#### Saving to .csv file 

In [15]:
import nltk

In [14]:
location_list = tweets_df.loc[tweets_df[6] != ''][6].values.tolist()
location_list

['日本 沖縄',
 'Berlin, Germany',
 'California, USA',
 '日本 東京',
 'Amsterdam, Istanbul, NYC',
 'Romeo & Juliette Laser Hair Removal Capital',
 '1 AU',
 'Roma',
 'New Jersey, USA',
 'New York, NY',
 'South Africa & Europe',
 'magicalmysteryplanttour.group',
 'Cleveland, OH',
 '東京都',
 'Panama',
 'Longview Texas',
 '$TSLAQ’s worst nightmare',
 'Boston, MA',
 '小高い丘の上',
 'Traveling ',
 'Chicago, IL',
 'Switzerland',
 'Baton Rouge, LA',
 'Chicago, IL',
 'Mars',
 'British Columbia, Canada',
 'New York, NY',
 'New York, NY',
 'Colorado Springs, CO',
 'Balboa Peninsula, SoCal USA',
 'Fremont, CA',
 'Orlando',
 'Vancouver Canada',
 '日吉→三田→丸の内/丸の外orTwitterLand ',
 'Lawrenceville, GA',
 'アメリカ中西部',
 'USA 🇺🇸',
 'Montréal, Québec',
 '地球',
 'Switzerland',
 'Chicago',
 'Montréal, Québec',
 'Los Angeles',
 'Fremont, CA',
 'San Francisco ',
 'Fremont, CA',
 'Fremont, CA',
 'en tu corazon',
 'Washington, DC',
 'Virginia',
 'Washington, DC',
 'Chicago, IL',
 'Irvine, CA',
 'New York, NY',
 'Hollywood, Los Angel

In [60]:
freq_dist = nltk.FreqDist(location_list)
freq_dist

FreqDist({'28054': 74, 'Chicago, IL': 70, 'United States': 67, 'Los Angeles, CA': 45, 'New York, NY': 39, 'Boston': 31, 'Amsterdam, Istanbul, NYC': 30, 'USA': 29, 'Hollywood': 29, 'Pompton Plains, New Jersey': 29, ...})

In [61]:
geo_freq = pd.DataFrame.from_dict(dict(freq_dist),orient='index',columns=['freq'])
geo_freq = geo_freq.sort_values('freq',ascending=False)
geo_freq

,freq
28054,74
"Chicago, IL",70
United States,67
"Los Angeles, CA",45
"New York, NY",39
...,...
US of A,1
Somewhere East of Suez,1
Somewhere,1
Europe,1


In [62]:
geo_freq.reset_index(inplace=True)
geo_freq = geo_freq.rename(columns = {'index':'loc'})
geo_freq = geo_freq.loc[geo_freq.freq >= 5]

In [63]:
import geopandas
from geopy.geocoders import Nominatim

In [64]:
locator = Nominatim(user_agent="myGeocoder")
geo_freq['location'] = geo_freq['loc'].apply(locator.geocode)

In [66]:
geo_freq.head(20)

,loc,freq,location
0,28054,74,"(Carabanchel, Madrid, Área metropolitana de Ma..."
1,"Chicago, IL",70,"(Chicago, Cook County, Illinois, United States..."
2,United States,67,"(United States, (39.7837304, -100.4458825))"
3,"Los Angeles, CA",45,"(Los Angeles, Los Angeles County, California, ..."
4,"New York, NY",39,"(New York, United States of America, (40.71272..."
5,Boston,31,"(Boston, Suffolk County, Massachusetts, United..."
6,"Amsterdam, Istanbul, NYC",30,None
7,USA,29,"(United States, (39.7837304, -100.4458825))"
8,Hollywood,29,"(Hollywood, Los Angeles, Los Angeles County, C..."
9,"Pompton Plains, New Jersey",29,"(Pompton Plains, Pequannock Township, Morris C..."


In [67]:
tags = api.trends_place(23424977) 
tags

[{'trends': [{'name': '#MarchForTrump',
    'url': 'http://twitter.com/search?q=%23MarchForTrump',
    'promoted_content': None,
    'query': '%23MarchForTrump',
    'tweet_volume': 67548},
   {'name': '#MAGAMillionMarch',
    'url': 'http://twitter.com/search?q=%23MAGAMillionMarch',
    'promoted_content': None,
    'query': '%23MAGAMillionMarch',
    'tweet_volume': 33482},
   {'name': '#SaturdayMorning',
    'url': 'http://twitter.com/search?q=%23SaturdayMorning',
    'promoted_content': None,
    'query': '%23SaturdayMorning',
    'tweet_volume': 10698},
   {'name': '#BidenHarrisVictoryDay',
    'url': 'http://twitter.com/search?q=%23BidenHarrisVictoryDay',
    'promoted_content': None,
    'query': '%23BidenHarrisVictoryDay',
    'tweet_volume': 19427},
   {'name': '#MillionMAGAMarch',
    'url': 'http://twitter.com/search?q=%23MillionMAGAMarch',
    'promoted_content': None,
    'query': '%23MillionMAGAMarch',
    'tweet_volume': 196193},
   {'name': 'Lilo',
    'url': 'http://tw